## Using Gemma without RAG


### Step 1: Install libraries

In [ ]:
!pip install -q langchain transformers torch
!pip install langchain-community --quiet
!pip install langchain_huggingface --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00


### Step 2: Import Packages

In [ ]:
#from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import notebook_login
import torch
import os
from google.colab import userdata

In [ ]:
# Set up the Hugging Face Hub API token and login to huggingface_hub
os.environ['HUGGINGFACE_HUB_TOKEN'] = userdata.get('HUGGINGFACE_HUB_TOKEN')
notebook_login()

In [ ]:
# Configuration
MODEL_NAME = "google/gemma-2b-it"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7
)

Device set to use cuda:0


In [ ]:
# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
user_input = input("\nYou: ")

# Format prompt for Gemma
prompt = f"<start_of_turn>user\n{user_input}<end_of_turn>\n<start_of_turn>model\n"

# Generate response
response = llm.invoke(prompt)

# Extract just the model's response
if isinstance(response, list):
    response = response[0]['generated_text']

# Clean up the output
response = response.split("<start_of_turn>model\n")[-1].strip()
print(f"\nAssistant: {response}")


You: How sex is determined?

Assistant: Sex is determined by a combination of genetic and environmental factors.

**Genetic factors:**

* **Sex chromosomes:** Humans have two sex chromosomes, X and Y.
* **XY chromosomal constitution:** Individuals with an X and Y chromosome are male.
* **XX chromosomal constitution:** Individuals with two X chromosomes are female.

**Environmental factors:**

* **Hormones:** Sex hormones, such as testosterone and estrogen, play a significant role in determining sex development and characteristics.
* **Environment:** Exposure to certain substances, such as hormones or toxins, can influence sex development.

**Development of sexual characteristics:**

* **Primary sexual characteristics:** These develop in the womb before birth, including the genitals, reproductive organs, and secondary sexual characteristics.
* **Secondary sexual characteristics:** These develop after birth and include changes in the breasts, hips, and body hair.

**It's important to no

## Using Gemma with RAG

### Step 0: Install additional packages

In [ ]:
!pip install -q -U chromadb pypdf sentence-transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.7 MB/s et

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
# ======================
# CONFIGURATION
# ======================
PDF_FOLDER_PATH = "/content/drive/My Drive/Book_Class_10_Ch8/"  # Folder containing your PDFs
CHROMA_DB_PATH = "/content/drive/MyDrive/"  # Where to store ChromaDB
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"

In [ ]:
# Create directories if they don't exist
os.makedirs(CHROMA_DB_PATH, exist_ok=True)

### Step 1: Data Preparation

*   Read text from PDF documents
*   Divide PDF document into smaller chunks  

In [ ]:
# ======================
# LOAD AND CHUNK PDFs
# ======================
def load_and_chunk_pdfs(pdf_folder):
    documents = []

    # Load each PDF in the folder
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            print(f"Processing {pdf_file}...")

            # Load PDF
            loader = PyPDFLoader(pdf_path)
            pages = loader.load()

            # Split into chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len
            )
            chunks = text_splitter.split_documents(pages)
            documents.extend(chunks)

    print(f"\nTotal documents: {len(documents)}")
    return documents

### Step 2: Create VectorStore

In [ ]:
# ======================
# CREATE VECTORSTORE
# ======================
def create_vectorstore(documents):
    # Initialize embedding function
    embedding_function = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={"device": DEVICE},
    )

    # Create and persist ChromaDB
    vector_db = Chroma.from_documents(
        documents=documents,
        embedding=embedding_function,
        persist_directory=CHROMA_DB_PATH
    )
    vector_db.persist()
    print(f"Vector store created at {CHROMA_DB_PATH}")
    return vector_db


In [ ]:
def load_vectorstore():
    # Initialize the same embedding function used during creation
    embedding_function = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={"device": DEVICE},
    )

    # Load the persisted database
    vector_db = Chroma(
        persist_directory=CHROMA_DB_PATH,
        embedding_function=embedding_function
    )

    print(f"Vector store loaded from {CHROMA_DB_PATH}")
    return vector_db

### Step 3: Setup Gemma Q&A Chain


*   Get access tokens from Hugging face
*   Initialize the tokenizer with the model.
*   Create a text generation pipeline.
*   Initialize the LLM with pipeline and model arguments
*   The final step is to generate the answers using both the vector store and the LLM.

It will generate embeddings to the input query or question retrieve the context from the vector store, and feed this to the LLM to generate the answers

In [ ]:
# ======================
# SETUP GEMMA QA CHAIN
# ======================
def setup_qa_chain(vector_db):
    # Load Gemma model
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    )

    # Create pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )

    # Wrap in LangChain
    llm = HuggingFacePipeline(pipeline=pipe)

    # Custom prompt for Gemma
    prompt_template = """<start_of_turn>user
    Use the following context to answer the question. Be concise and accurate.
    If you don't know, say you don't know. Don't make up answers.

    Context: {context}

    Question: {question}<end_of_turn>
    <start_of_turn>model
    """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # Create QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

    return qa_chain

### Step 4: Main execution

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.flush_and_unmount()  # Completely unmount
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
def vectorstore_exists():
    return os.path.exists(CHROMA_DB_PATH)

In [ ]:
# ======================
# MAIN EXECUTION
# ======================

# Step 1: Process PDFs
print("Loading and chunking PDFs...")
documents = load_and_chunk_pdfs(PDF_FOLDER_PATH)

# Step 2: Create vector store
print("\nCreating vector store...")
# if vectorstore_exists():
#   vector_db = load_vectorstore()
# else:
vector_db = create_vectorstore(documents)

# Step 3: Setup QA system
print("\nSetting up QA system...")
qa_chain = setup_qa_chain(vector_db)


Loading and chunking PDFs...
Processing jesc108.pdf...

Total documents: 22

Creating vector store...


<ipython-input-14-2d2819c96bb2>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-14-2d2819c96bb2>:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


Vector store created at /content/drive/MyDrive/

Setting up QA system...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
# question answering

question = input("\nYour question: ")
result = qa_chain.invoke({"query": question})
response = result["result"].split("<start_of_turn>model\n")[-1].strip()
print("\nAnswer:", response)

# print("\nAnswer:", result["result"])



# Show sources
print("\nSources:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\nSource {i}:")
    print("Content:", doc.page_content[:200] + "...")
    print("From:", doc.metadata.get("source", "Unknown"))


Your question: How sex is determined?

Answer: According to the passage, in human beings, the sex of the individual is largely genetically determined. The genes inherited from our parents decide whether we will be boys or girls.

Sources:

Source 1:
Content: inheritance might work?
8.2.4 Sex Determination
We have discussed the idea that the two sexes participating in sexual
reproduction must be somewhat different from each other for a number
of reasons. H...
From: /content/drive/My Drive/Book_Class_10/jesc108.pdf

Source 2:
Content: inheritance might work?
8.2.4 Sex Determination
We have discussed the idea that the two sexes participating in sexual
reproduction must be somewhat different from each other for a number
of reasons. H...
From: /content/mydrive/My Drive/Book_Class_10_Ch8/jesc108.pdf

Source 3:
Content: inheritance might work?
8.2.4 Sex Determination
We have discussed the idea that the two sexes participating in sexual
reproduction must be somewhat different from each other 